In [1]:
import pandas as pd
import subprocess
from pathlib import Path
import tarfile
import arff

In [ ]:
# Pfad zur .Z Datei und Zielverzeichnis
archive_path = Path('diabetes\\diabetes-data.tar.Z')
extract_dir = Path('diabetes\\extracted_data')

# Erstelle das Zielverzeichnis, falls es nicht existiert
extract_dir.mkdir(parents=True, exist_ok=True)

# Entpacke die .Z Datei mit dem 'uncompress' Tool
subprocess.run(["C:\\Program Files\\7-Zip\\7z.exe", 'e', str(archive_path), f'-o{extract_dir}'])

# Haupt-DataFrame für alle Patienten
main_df = pd.DataFrame(columns=['patient_id', 'date', 'time', 'code', 'value'])

# Alle entpackten Dateien im Verzeichnis durchlaufen
for file in extract_dir.iterdir():
    df = pd.DataFrame(columns=['date', 'time', 'code', 'value'])
    with open(file, 'r') as data:
        for line in data.readlines():
            line = line.strip()  # Entferne Leerzeichen und Zeilenumbrüche
            line_split = line.split('\t')
            line_df = pd.DataFrame([line_split], columns=['date', 'time', 'code', 'value'])
            df = pd.concat([df, line_df], ignore_index=True)

    # ID aus dem Dateinamen ableiten
    patient_id = file.stem[-2:]
    df['patient_id'] = patient_id
    main_df = pd.concat([main_df, df], ignore_index=True)

# Speichere den zusammengeführten DataFrame als CSV
main_df.to_csv(extract_dir / 'diabetes_data_all_patients.csv', index=False)
print('Alle Dateien wurden zusammengeführt und gespeichert.')


In [2]:
# Pfad zur .tar Datei und das Ziel für die ARFF-Datei
tar_path = 'diabetes\extracted_data\diabetes-data.tar'
arff_path = 'diabetes_data_all_patients.arff'

# Leerer DataFrame für die Daten
data = pd.DataFrame(columns=['patient_id', 'date', 'time', 'code', 'value'])

# Entpacke die .tar Datei und lese jede Datei in den DataFrame
with tarfile.open(tar_path, 'r') as tar:
    for member in tar.getmembers():
        if member.isfile():  # Nur Dateien lesen
            f = tar.extractfile(member)
            if f is not None:
                temp_df = pd.read_csv(f, delimiter='\t', names=['date', 'time', 'code', 'value'])
                temp_df['patient_id'] = member.name.split('/')[-1].split('.')[0]  # Verwende den Dateinamen als ID
                data = pd.concat([data, temp_df], ignore_index=True)

# Konvertiere nicht-numerische Werte in den Spalten 'patient_id', 'code', und 'value' zu NaN
#data['patient_id'] = pd.to_numeric(data['patient_id'], errors='coerce')
data['code'] = pd.to_numeric(data['code'], errors='coerce')
data['value'] = pd.to_numeric(data['value'], errors='coerce')

# Entferne Zeilen mit NaN-Werten in den numerischen Spalten
data = data.dropna(subset=['patient_id', 'code', 'value','date'])

# ARFF-Datenstruktur erstellen
arff_data = {
    'description': '',
    'relation': 'diabetes_data',
    'attributes': [
        ('patient_id', 'STRING'),
        ('date', 'STRING'),  # Behandle das Datum als STRING, damit WEKA es akzeptiert
        ('time', 'STRING'),  # Behandle die Zeit ebenfalls als STRING
        ('code', 'NUMERIC'),
        ('value', 'NUMERIC')
    ],
    'data': data.values.tolist(),
}

# Speichere die ARFF-Datei
with open(arff_path, 'w') as f:
    arff.dump(arff_data, f)

print("ARFF-Datei wurde erfolgreich erstellt.")

ARFF-Datei wurde erfolgreich erstellt.
